#Question: For specific political actors, who is their most associated person during SARS-CoV-2 outbreak in their home countries media 

-Total time to run around 10 - 12 minutes

##download dependencies needed

In [0]:
#library and code setup - retrived from Sample GDELT starter only.ipynb "Written by James Atlas"
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
# !pip install newspaper3k
!pip install gdelt
# !pip install -U -q PyDrive
#library and code setup
import pyspark, os
from pyspark import SparkConf, SparkContext

os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

In [0]:

import sys, os
from operator import add
import time
import timeit #used for timing project for scalability
start = timeit.default_timer() #time it used for checking scalability 
os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

##Remove files that I previously had for fresh start

In [0]:
#removes files from previous runs and creates Completed_files directory
!rm -rf articles
!rm *.csv
!rm -rf Completed_files 
!mkdir Completed_files
!rm -rf table_values 
!rm -rf *.TXT
!rm -rf *.TXT.*


rm: cannot remove '*.csv': No such file or directory


#Get all the data required from the GKG table using PlPy - takes about 1 minute per month - time taken '2019-12-31','2020-05-29' for during covid period - start date of covid to current date with covid still relevant

In [0]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='gkg',coverage=False) #not updata at 15mins #I've tried events now its gkg same result :( no matches
          d.to_csv("/content/table_values/" + filename,encoding='utf-8',index=False)
    except:
        print("Error occurred")


!mkdir table_values

# pull the data from gdelt into multi files; this may take a long time
dates = [get_filename(x) for x in pd.date_range('2019-12-31','2020-05-29')] #this time period is from the start of covid 19 outbreak in nz for 4 months period '2019-12-31','2020-04-30'

results = list(e.map(intofile,dates))


def caf(path, foldername):
  from pyspark.sql import SQLContext
  sqlContext = SQLContext(sc)
  directory = "/content/"+path+"/*.csv"
  combined_files = sqlContext.read.csv(directory, header=True)
  combined_files.write.csv("/content/Completed_files/" + foldername + ".csv", header=True)   #header=True
  print("Combined: " + str(combined_files.count()) + " Files")
  return combined_files




combined_files = caf('/table_values/', 'table_values')





/usr/local/lib/python3.6/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did not return data for date time 20200117234500
  warnings.warn(message)


Error occurred


/usr/local/lib/python3.6/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did not return data for date time 20200203234500
  warnings.warn(message)


Error occurred


/usr/local/lib/python3.6/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did not return data for date time 20200208234500
  warnings.warn(message)


...

Error occurred


/usr/local/lib/python3.6/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did not return data for date time 20200513234500
  warnings.warn(message)


Error occurred


/usr/local/lib/python3.6/dist-packages/gdelt/parallel.py:103: UserWarning: GDELT did not return data for date time 20200517234500
  warnings.warn(message)


Error occurred
Combined: 271330 Files


###Zip all the files from GKG this is almsot a gig of data for 3 months don't do this unless really needed

In [0]:
#used for checking file sizes
#combined_files.count() #counts the number of files

271330

In [0]:
#Zips all the data taken from the GKG database before any changes have been made to it. This is a large file around 1 gig per 3 months so only use if necessary!
# !zip -r /content/Completed_files.zip /content/Completed_files #only use if you want all the data

In [0]:
start2 = timeit.default_timer()

##Drop unwanted data to keep smaller processing time and cleaner code - This is done at this stage to avoid getting head errors 

In [0]:
Persons_Source_dataframe = combined_files.select("Persons", "SourceCommonName")
Persons_Source_dataframe.show() #shows what data we currently have
Persons_Source_dataframe = Persons_Source_dataframe.toPandas() #changed to pandas dataframe as easier to minipulate later 

+--------------------+--------------------+
|             Persons|    SourceCommonName|
+--------------------+--------------------+
|john oconnor;jess...|     therepublic.com|
|                null|            wlfi.com|
|russell pigg;stev...| quadcitiesdaily.com|
|scott harris;john...|    cullmantimes.com|
|                null|dailyexaminer.com.au|
|megan miller;capr...|     hailvarsity.com|
|thomas branch;sio...|chestertontribune...|
|hashem bathayi go...|     kuwaittimes.net|
|aldine isds;steve...|           chron.com|
|kay ivey;donald t...|    gadsdentimes.com|
|                null|            klll.com|
|deforest buckner;...|        indystar.com|
|                null|      theisonews.com|
|scott asalone;mar...|   nbcwashington.com|
|kelli segers;kayl...|          iheart.com|
|mike dewine;patri...|            wtma.com|
|jacqueline jackie...|            wbnq.com|
|jacqueline jackie...|        mystar98.com|
|                null|   wellandtribune.ca|
|donald trump;wayn...|fortworthb

##Get all the countires by the domain of the URL 

In [0]:
#Download the domains by country file needed for processing next step
!wget http://data.gdeltproject.org/supportingdatasets/DOMAINSBYCOUNTRY-ENGLISH.TXT
csv_file = "DOMAINSBYCOUNTRY-ENGLISH.TXT"

--2020-05-31 05:18:14--  http://data.gdeltproject.org/supportingdatasets/DOMAINSBYCOUNTRY-ENGLISH.TXT
Resolving data.gdeltproject.org (data.gdeltproject.org)... 108.177.112.128, 2607:f8b0:4001:c12::80
Connecting to data.gdeltproject.org (data.gdeltproject.org)|108.177.112.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 426990 (417K) [text/plain]
Saving to: ‘DOMAINSBYCOUNTRY-ENGLISH.TXT’

DOMAINSBYCOUNTRY-EN 100%[===================>] 416.98K  --.-KB/s    in 0.003s  

2020-05-31 05:18:15 (145 MB/s) - ‘DOMAINSBYCOUNTRY-ENGLISH.TXT’ saved [426990/426990]



In [0]:
def process_source_country(csv_file):
    source_dict = {}
    with open(csv_file) as f:
        lines = f.readlines()
        for line in lines:
            try:
                source, code, country = line.split(None, maxsplit=2)
                source_dict[source] = code
            except ValueError:
                source, code = line.split(None, maxsplit=1)
                source_dict[source] = code
    return source_dict
source_country_dict = process_source_country(csv_file)
sources = sc.broadcast(source_country_dict)

In [0]:
def define_country(x):
    """Helper function takes the SourceCommonName and looks up in dictionary returning the Country"""
    if x in sources.value:
      return sources.value[x]

Persons_Source_dataframe['Country'] = Persons_Source_dataframe['SourceCommonName'].apply(lambda x: define_country(x)) #Makes a new column called country and collects the associated country via the domain name function
Persons_Source__country_dataframe = Persons_Source_dataframe #Change the name to fit the columns
Persons_Source__country_dataframe.head(5)

,Persons,SourceCommonName,Country
0,john oconnor;jessica basham;john patterson,therepublic.com,US
1,None,wlfi.com,US
2,russell pigg;steve holt;sheffield helen keller...,quadcitiesdaily.com,None
3,scott harris;john atkinson,cullmantimes.com,US
4,None,dailyexaminer.com.au,AS


###Clean up data and drop unwanted tables

In [0]:
Persons_Source__country_dataframe.dropna(subset = ["Persons"], inplace=True) #Drop all the rows with no Nan or None in persons column
Persons_Source__country_dataframe.dropna(subset = ["Country"], inplace=True) #Drop all the rows with no Nan or None in Country column
Cleaned_processed_data = Persons_Source__country_dataframe[['Persons', 'Country']] # keep only the data that is relevant 
Cleaned_processed_data.count() 

Persons    157018
Country    157018
dtype: int64

In [0]:
Cleaned_processed_data.head(5) #see what the current data looks like

,Persons,Country
0,john oconnor;jessica basham;john patterson,US
3,scott harris;john atkinson,US
6,thomas branch;sionna hartigan;tamsyn muir;abbe...,US
7,hashem bathayi golpayegani;kianoush jahanpour;...,KU
8,aldine isds;steve gonzales;haryie garrigues;el...,US


### Save cleaned data to CSV file so able to run from this point on without running prior downloads and cleaning data.

In [0]:
Cleaned_processed_data.to_csv(r'Cleaned_processed_data.csv', index=False, header=True) #save the entire file

In [0]:
#split the files by country to have smaller files and drop the country column
Cleaned_processed_data_nz = Cleaned_processed_data[Cleaned_processed_data.Country.str.contains('NZ',case=False)] #Only keep columns with NZ  etc..
Cleaned_processed_data_nz = Cleaned_processed_data_nz['Persons'] # keep only the data that is relevant 

Cleaned_processed_data_aus = Cleaned_processed_data[Cleaned_processed_data.Country.str.contains('AS',case=False)] 
Cleaned_processed_data_aus = Cleaned_processed_data_aus['Persons'] # keep only the data that is relevant 

Cleaned_processed_data_us = Cleaned_processed_data[Cleaned_processed_data.Country.str.contains('US',case=False)] 
Cleaned_processed_data_us = Cleaned_processed_data_us['Persons'] # keep only the data that is relevant 

Cleaned_processed_data_uk = Cleaned_processed_data[Cleaned_processed_data.Country.str.contains('UK',case=False)] 
Cleaned_processed_data_uk = Cleaned_processed_data_uk['Persons'] # keep only the data that is relevant 


Cleaned_processed_data_jp = Cleaned_processed_data[Cleaned_processed_data.Country.str.contains('JA',case=False)]
Cleaned_processed_data_jp = Cleaned_processed_data_jp['Persons'] # keep only the data that is relevant 


Cleaned_processed_data_china = Cleaned_processed_data[Cleaned_processed_data.Country.str.contains('CH',case=False)] 
Cleaned_processed_data_china = Cleaned_processed_data_china['Persons'] # keep only the data that is relevant 


Cleaned_processed_data_fiji = Cleaned_processed_data[Cleaned_processed_data.Country.str.contains('FJ',case=False)] 
Cleaned_processed_data_fiji = Cleaned_processed_data_fiji['Persons'] # keep only the data that is relevant 


#This next step is not needed if you do not wish to save the data for use later

In [0]:
#Saves each individual countries dataframe to CSV files 
Cleaned_processed_data_nz.to_csv(r'Cleaned_processed_data_nz.csv', index=False, header=True) #save only New zealands data to CSV file
Cleaned_processed_data_aus.to_csv(r'Cleaned_processed_data_aus.csv', index=False, header=True) #Australia
Cleaned_processed_data_us.to_csv(r'Cleaned_processed_data_us.csv', index=False, header=True) #United States
Cleaned_processed_data_uk.to_csv(r'Cleaned_processed_data_uk.csv', index=False, header=True) #United Kingdom
Cleaned_processed_data_jp.to_csv(r'Cleaned_processed_data_jp.csv', index=False, header=True) #japan
Cleaned_processed_data_china.to_csv(r'Cleaned_processed_data_china.csv', index=False, header=True) #China
Cleaned_processed_data_fiji.to_csv(r'Cleaned_processed_data_fiji.csv', index=False, header=True) #Fiji

#Load the data from CSV File if required


In [0]:
Cleaned_processed_data_nz = sc.textFile("Cleaned_processed_data_nz.csv") #loads the CSV file in to a pyspark RDD for each country
Cleaned_processed_data_aus = sc.textFile("Cleaned_processed_data_aus.csv")
Cleaned_processed_data_us = sc.textFile("Cleaned_processed_data_us.csv")
Cleaned_processed_data_uk = sc.textFile("Cleaned_processed_data_uk.csv")
Cleaned_processed_data_jp = sc.textFile("Cleaned_processed_data_jp.csv")
Cleaned_processed_data_china = sc.textFile("Cleaned_processed_data_china.csv")
Cleaned_processed_data_fiji = sc.textFile("Cleaned_processed_data_fiji.csv")

#Begin Processing data this is dependent on what country/data you pass in- loads the data from CSV file saved prior - No needed to run this if you ran data collection prior

In [0]:
#Split all values up and assign value of 1 in tuple format (Name, 1) looks like [('Persons,Country', 1), ('mike davis', 1), ('jason day', 1), ('bob charles', 1), ('dustin johnson', 1), ('jack nicklaus', 1), ('thomas pagel,NZ',
def assign_value_to_pair(file):
  value_pair = file.flatMap(lambda line: [(item, 1) for item in line.strip().split(";")])
  return value_pair

In [0]:
#Combine all people to get counts for market basket use later with A-Priori checking they are greater than a specified value for relevance 
def count_all_people(pairs):
  all_people_sum = pairs.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False) 
  all_people_sum.take(5)
  return all_people_sum

#Begin Applying Algorithms 

#Implement A-Priori/Market Basket

In [0]:
def relevant_tuples(people, level):
  greater_than_100 = people.filter(lambda x: x[1] >= level)
  broadcastVar = sc.broadcast(greater_than_100.collectAsMap()) #broadcast with only the strings
  return broadcastVar


In [0]:
def pair_sort(sub_list, broadcast):
  sub_list = sub_list.strip().split(";")
  new_list = []
  for index, i in enumerate(sub_list):
    for index2, j in enumerate(sub_list):
       if index2 < index:
          if (i in broadcast.value and j in broadcast.value):
            new_list.append(tuple(sorted((i, j))))   #
  return new_list


In [0]:
def division(ele1, ele2, num, broadcastVar):
  """Helper function for market basket analysis """
  total = broadcastVar.value[ele1]
  return(((ele1, ele2), num/total)) #total is the value held in the greater_than_100 broadcast


In [0]:
def Create_confidence(file, level_of_tolerance):
  value_pair = assign_value_to_pair(file)
  all_people_sum = count_all_people(value_pair)
  broadcastVar = relevant_tuples(all_people_sum, level_of_tolerance)
  pairs_of_people = file.flatMap(lambda line: (pair_sort(line, broadcastVar)))

  pairs_of_people_tuple = pairs_of_people.map(lambda x: (x, 1))
  total_pairs_of_people = pairs_of_people_tuple.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], False) 


  # #Calculat the confidence score left side of tuple and right side of tuple
  
  right_basket = total_pairs_of_people.map(lambda x: division(x[0][0], x[0][1], x[1], broadcastVar))
  left_basket = total_pairs_of_people.map(lambda x: division(x[0][1], x[0][0], x[1], broadcastVar))
  confidence_score = right_basket.union(left_basket).sortBy(lambda x: x[1], False) 


  return confidence_score


In [0]:
#Calculate the confidence for each Country 
nz_confidence = Create_confidence(Cleaned_processed_data_nz, 10)
aus_confidence = Create_confidence(Cleaned_processed_data_aus, 3)
us_confidence = Create_confidence(Cleaned_processed_data_us, 100)
uk_confidence = Create_confidence(Cleaned_processed_data_uk, 100)
jp_confidence = Create_confidence(Cleaned_processed_data_jp, 3)
china_confidence = Create_confidence(Cleaned_processed_data_china, 3)
fiji_confidence = Create_confidence(Cleaned_processed_data_fiji, 3)

In [0]:
dbg(nz_confidence)

[(('joe biden', 'donald trump'), 1.0), (('meghan markle', 'prince harry'), 0.8), (('andrew cuomo', 'donald trump'), 0.8), (('prince harry', 'meghan markle'), 0.6153846153846154), (('andrew cuomo', 'joe biden'), 0.5), (('scott morrison', 'jacinda ardern'), 0.45454545454545453), (('joe biden', 'andrew cuomo'), 0.45454545454545453), (('boris johnson', 'donald trump'), 0.4375), (('ashley bloomfield', 'jacinda ardern'), 0.42857142857142855), (('david clark', 'jacinda ardern'), 0.3333333333333333), (('andrew cuomo', 'boris johnson'), 0.3), (('grant robertson', 'jacinda ardern'), 0.2727272727272727), (('donald trump', 'joe biden'), 0.22916666666666666), (('ashley bloomfield', 'david clark'), 0.21428571428571427), (('david clark', 'grant robertson'), 0.2), (('david clark', 'ashley bloomfield'), 0.2), (('boris johnson', 'andrew cuomo'), 0.1875), (('joe biden', 'boris johnson'), 0.18181818181818182), (('shane jones', 'jacinda ardern'), 0.18181818181818182), (('donald trump', 'andrew cuomo'), 0.1

In [0]:
#Define all the people I want to look at per country
nz_pols = ['jacinda ardern',  'ashley bloomfield', 'david clark']
aus_pols = ['scott morrison', 'greg hunt']
us_pols=['donald trump', 'hillary clinton']
uk_pols =['boris johnson', 'queen elizabeth', 'matt hancock']
japan_pols = ['shinzo abe', 'katsunobu kato']  
china_pols = ['xi jinping', 'ma xiaowei']
fiji_pols = ['frank bainimarama', 'ifereimi waqainabete']

In [0]:
def calculate_confidence_per_person(pol_names, country_confidence):
  Confidence_combined = {}
  for j in country_confidence.collect(): #can use collect here as its a small amount of data its not relevant to speed of the project
    name = j[0][0]
    if name in pol_names:
      Confidence_combined.setdefault(name,[]).append((j[0][1], j[1]))
  return Confidence_combined


#get the people we are interested in for each country
nz_actors_confidence_corona = calculate_confidence_per_person(nz_pols, nz_confidence)
aus_actors_confidencee_corona = calculate_confidence_per_person(aus_pols, aus_confidence)
us_actors_confidencee_corona = calculate_confidence_per_person(us_pols, us_confidence)
uk_actors_confidencee_corona = calculate_confidence_per_person(uk_pols, uk_confidence)
jp_actors_confidencee_corona = calculate_confidence_per_person(japan_pols, jp_confidence)
china_actors_confidencee_corona = calculate_confidence_per_person(china_pols, china_confidence)
fiji_actors_confidencee_corona = calculate_confidence_per_person(fiji_pols, fiji_confidence)


In [0]:
dbg(nz_actors_confidence_corona) 
dbg(aus_actors_confidencee_corona) 
dbg(us_actors_confidencee_corona) 
dbg(uk_actors_confidencee_corona) 
dbg(jp_actors_confidencee_corona) 
dbg(china_actors_confidencee_corona) 
dbg(fiji_actors_confidencee_corona) 

{'ashley bloomfield': [('jacinda ardern', 0.42857142857142855), ('david clark', 0.21428571428571427), ('grant robertson', 0.14285714285714285), ('donald trump', 0.07142857142857142)], 'david clark': [('jacinda ardern', 0.3333333333333333), ('grant robertson', 0.2), ('ashley bloomfield', 0.2), ('scott morrison', 0.06666666666666667)], 'jacinda ardern': [('ashley bloomfield', 0.08955223880597014), ('grant robertson', 0.08955223880597014), ('scott morrison', 0.07462686567164178), ('david clark', 0.07462686567164178), ('donald trump', 0.04477611940298507), ('shane jones', 0.029850746268656716), ('meghan markle', 0.014925373134328358), ('prince harry', 0.014925373134328358)]}
{'greg hunt': [('scott morrison', 0.5076923076923077), ('brendan murphy', 0.4), ('donald trump', 0.16923076923076924), ('simon birmingham', 0.15384615384615385), ('paul kelly', 0.13846153846153847), ('andrew cuomo', 0.13846153846153847), ('tedros adhanom ghebreyesus', 0.1076923076923077), ('gladys berejiklian', 0.10769

#Results No relevant data was able to be found for china and fiji with limited data from japan there for china and fiji dropped for final result

In [0]:

def get_highest(element):
  highest_confidence_pair = []
  for name, confidence in element.items():
      highest = 0 
      highest_name = ""
      # print(name, confidence)
      for name2, value in confidence:
        if value > highest:
          highest = value
          highest_name = name2
      highest_confidence_pair.append((name ,highest_name)) 
  return highest_confidence_pair

nz_highest = get_highest(nz_actors_confidence_corona)
aus_highest = get_highest(aus_actors_confidencee_corona)
us_highest = get_highest(us_actors_confidencee_corona)
uk_highest = get_highest(uk_actors_confidencee_corona)
jp_highest = get_highest(jp_actors_confidencee_corona)
all_counties = [(nz_highest, "New Zealand"), (aus_highest, "Australia"), 
                (us_highest, "United States"), (uk_highest, "United Kingdom"), (jp_highest, "Japan")]

for i in all_counties:
  print(i)

([('ashley bloomfield', 'jacinda ardern'), ('david clark', 'jacinda ardern'), ('jacinda ardern', 'ashley bloomfield')], 'New Zealand')
([('greg hunt', 'scott morrison'), ('scott morrison', 'brendan murphy')], 'Australia')
([('hillary clinton', 'donald trump'), ('donald trump', 'joe biden')], 'United States')
([('matt hancock', 'boris johnson'), ('boris johnson', 'matt hancock')], 'United Kingdom')
([('shinzo abe', 'yuriko koike')], 'Japan')


In [0]:
for country, country_name in all_counties:
  print("For " + country_name)
  for name, link in country:
    print("The most associated person with " + name + " is " + link)


For New Zealand
The most associated person with ashley bloomfield is jacinda ardern
The most associated person with david clark is jacinda ardern
The most associated person with jacinda ardern is ashley bloomfield
For Australia
The most associated person with greg hunt is scott morrison
The most associated person with scott morrison is brendan murphy
For United States
The most associated person with hillary clinton is donald trump
The most associated person with donald trump is joe biden
For United Kingdom
The most associated person with matt hancock is boris johnson
The most associated person with boris johnson is matt hancock
For Japan
The most associated person with shinzo abe is yuriko koike


In [0]:
stop = timeit.default_timer()
print('Time taken to download all files inclusive of download: ', stop - start) 
print('Time taken to download all files exclusive of download: ', stop - start2) 

Time taken to download all files inclusive of download:  335.0886744159998
Time taken to download all files exclusive of download:  53.71491021700058
